# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import haversine as hv
from datetime import timedelta

from sklearn.model_selection import train_test_split

In [3]:
full_data = pd.read_csv('data/train.csv')
weather = pd.read_csv('data/weather_cleaned_stack_back.csv')
spray = pd.read_csv('data/spray.csv')

In [4]:
full_data.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent'],
      dtype='object')

In [5]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null object
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.0+ KB


In [6]:
full_data.columns.to_list()

['Date',
 'Address',
 'Species',
 'Block',
 'Street',
 'Trap',
 'AddressNumberAndStreet',
 'Latitude',
 'Longitude',
 'AddressAccuracy',
 'NumMosquitos',
 'WnvPresent']

In [7]:
weather.drop(['Unnamed: 0'],axis=1,inplace=True)

In [8]:
weather.dtypes

Date            object
Sunset         float64
DaylightHrs    float64
Tmax           float64
Tmin           float64
Tavg           float64
Depart         float64
DewPoint       float64
WetBulb        float64
Cool           float64
PrecipTotal    float64
StnPressure    float64
SeaLevel       float64
ResultSpeed    float64
ResultDir      float64
AvgSpeed       float64
Station          int64
dtype: object

In [9]:
weather.Date = weather.Date.astype('datetime64[ns]')

In [10]:
weather['Year'] = weather.Date.map(lambda x : x.year)

In [11]:
weather.Year.value_counts()

2013    368
2011    368
2009    368
2007    368
2014    368
2012    368
2010    368
2008    368
Name: Year, dtype: int64

In [12]:
spray = spray.drop_duplicates(subset=['Latitude', 'Longitude'], keep=False)

In [13]:
spray.Date.value_counts()

2013-08-15    2668
2013-08-29    2302
2013-07-17    2202
2013-07-25    1607
2013-08-22    1587
2011-09-07    1571
2013-08-08    1195
2013-09-05     924
2013-08-16     141
2011-08-29      95
Name: Date, dtype: int64

## Feature Engineering

Distance to nearest spray point within 3 days of each day.

In [14]:
spray.Date = spray.Date.astype('datetime64[ns]')
full_data.Date = full_data.Date.astype('datetime64[ns]')
weather.Date = weather.Date.astype('datetime64[ns]')

In [15]:
weather.Date.head()

0   2007-05-01
1   2007-05-02
2   2007-05-03
3   2007-05-04
4   2007-05-05
Name: Date, dtype: datetime64[ns]

In [16]:
weather.Date.tail()

2939   2014-10-27
2940   2014-10-28
2941   2014-10-29
2942   2014-10-30
2943   2014-10-31
Name: Date, dtype: datetime64[ns]

In [17]:
full_data.Date.head()

0   2007-05-29
1   2007-05-29
2   2007-05-29
3   2007-05-29
4   2007-05-29
Name: Date, dtype: datetime64[ns]

We don't have trap / mosquito / wnv data for the years of even number. Let it be...it will be a left join for full_data with weather

In [18]:
full_data.Date.tail()

10501   2013-09-26
10502   2013-09-26
10503   2013-09-26
10504   2013-09-26
10505   2013-09-26
Name: Date, dtype: datetime64[ns]

In [19]:
spray.Date.tail()

14830   2013-09-05
14831   2013-09-05
14832   2013-09-05
14833   2013-09-05
14834   2013-09-05
Name: Date, dtype: datetime64[ns]

In [20]:
start_date = full_data.Date[7576]
end_date = start_date + timedelta(days=7)

In [21]:
mask = (spray.Date >= start_date) & (spray.Date <= end_date)
spray_subset = spray[mask]
spray_subset.shape[0]

1571

In [22]:
spray['coords'] = spray.apply(lambda x: ([x.Latitude, x.Longitude]), axis=1)

In [23]:
def nearest_spray(date, in_coords, n_diff=7):   
  
    start_date = date
    end_date = date + timedelta(days=n_diff)
    mask = (spray.Date >= start_date) & (spray.Date <= end_date)
    spray_subset = spray[mask]
    
    if not spray_subset.empty:
        dist = [hv.haversine(in_coords,coord) for coord in (spray_subset.coords)]     
        return round(min(dist),2)
    else:
        # print('empty_subset')
        return np.nan


In [24]:
%time

full_data['spray_dist'] = full_data.apply(lambda x: nearest_spray(x.Date,[x.Latitude, x.Longitude]), axis=1)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [25]:
## spray_status : 1 if less than 7 days and 3 km, else 0
full_data['spray_status'] = full_data.apply(lambda x: 1 if (x.spray_dist <= 3) else 0,axis=1)

In [26]:
full_data.spray_status.value_counts()

0    10208
1      298
Name: spray_status, dtype: int64

We can see that only 298 sites are covered under the spray conditions

In [27]:
full_data['Year'] = full_data.Date.map(lambda x : x.year)
spray['Year'] = spray.Date.map(lambda x: x.year)

In [28]:
spray.Year.value_counts()

2013    12626
2011     1666
Name: Year, dtype: int64

In [29]:
full_data.Year.value_counts()

2007    3811
2013    2392
2009    2249
2011    2054
Name: Year, dtype: int64

In [30]:
full_data.groupby(['Year'])['Trap'].nunique()

Year
2007    116
2009     73
2011     80
2013     74
Name: Trap, dtype: int64

In [31]:
df = pd.DataFrame(full_data.groupby(['Date','Address','Species',
                                     'Block','Street','Trap',
                                     'AddressNumberAndStreet',
                                     'Latitude','Longitude',
                                     'AddressAccuracy', 'WnvPresent'])['NumMosquitos'].sum())

In [32]:
df.reset_index(inplace=True)

In [33]:
df.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,WnvPresent,NumMosquitos
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,0,1
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,0,2
2,2007-05-29,"1100 South Peoria Street, Chicago, IL 60608, USA",CULEX RESTUANS,11,S PEORIA ST,T091,"1100 S PEORIA ST, Chicago, IL",41.862292,-87.648860,8,0,1
3,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",CULEX RESTUANS,11,W CHICAGO,T049,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,8,0,1
4,2007-05-29,"1500 North Long Avenue, Chicago, IL 60651, USA",CULEX RESTUANS,15,N LONG AVE,T153,"1500 N LONG AVE, Chicago, IL",41.907645,-87.760886,8,0,1


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8610 entries, 0 to 8609
Data columns (total 12 columns):
Date                      8610 non-null datetime64[ns]
Address                   8610 non-null object
Species                   8610 non-null object
Block                     8610 non-null int64
Street                    8610 non-null object
Trap                      8610 non-null object
AddressNumberAndStreet    8610 non-null object
Latitude                  8610 non-null float64
Longitude                 8610 non-null float64
AddressAccuracy           8610 non-null int64
WnvPresent                8610 non-null int64
NumMosquitos              8610 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 807.3+ KB


After 'collapsing' the rows by summing up the number of mosquitos, we reduced the number of rows from 10506 to 8610

In [35]:
full_data.drop(['NumMosquitos'],axis=1,inplace=True)

In [36]:
full_data = full_data.drop_duplicates()

In [37]:
full_data.shape[0]

8610

In [38]:
full_condensed = pd.merge(df, full_data, how='left',on=['Date','Address','Species',
                                     'Block','Street','Trap',
                                     'AddressNumberAndStreet',
                                     'Latitude','Longitude',
                                     'AddressAccuracy', 'WnvPresent'])

In [39]:
full_condensed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8610 entries, 0 to 8609
Data columns (total 15 columns):
Date                      8610 non-null datetime64[ns]
Address                   8610 non-null object
Species                   8610 non-null object
Block                     8610 non-null int64
Street                    8610 non-null object
Trap                      8610 non-null object
AddressNumberAndStreet    8610 non-null object
Latitude                  8610 non-null float64
Longitude                 8610 non-null float64
AddressAccuracy           8610 non-null int64
WnvPresent                8610 non-null int64
NumMosquitos              8610 non-null int64
spray_dist                1334 non-null float64
spray_status              8610 non-null int64
Year                      8610 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(6), object(5)
memory usage: 1.1+ MB


## Weather data Mapping

In [40]:
STATIONS = {
    1 : (41.995,-87.933),
    2 : (41.786,-87.752)
}

In [41]:
def nearest_station(in_coords):
    
    dist = {k:hv.haversine(in_coords,v) for k,v in STATIONS.items()}
    
    return min(dist, key=dist.get)


In [42]:
full_condensed['nearest_station'] = full_condensed.apply(lambda x: nearest_station([x.Latitude, x.Longitude]), axis=1)

In [43]:
full_condensed.nearest_station.value_counts()

2    6090
1    2520
Name: nearest_station, dtype: int64

In [44]:
full_condensed['date_lag'] = full_condensed.Date.map(lambda x : x - timedelta(days=7))

In [45]:
full_condensed = full_condensed.merge(weather,left_on=['date_lag','nearest_station'],right_on=['Date','Station'])

In [46]:
full_condensed.dtypes

Date_x                    datetime64[ns]
Address                           object
Species                           object
Block                              int64
Street                            object
Trap                              object
AddressNumberAndStreet            object
Latitude                         float64
Longitude                        float64
AddressAccuracy                    int64
WnvPresent                         int64
NumMosquitos                       int64
spray_dist                       float64
spray_status                       int64
Year_x                             int64
nearest_station                    int64
date_lag                  datetime64[ns]
Date_y                    datetime64[ns]
Sunset                           float64
DaylightHrs                      float64
Tmax                             float64
Tmin                             float64
Tavg                             float64
Depart                           float64
DewPoint        

In [47]:
full_condensed.drop(['Date_y','Year_y'],axis=1, inplace=True)

In [48]:
full_condensed.rename(columns={'Date_x': 'Date', 'Year_x': 'Year'},inplace=True)

In [52]:
full_condensed.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'WnvPresent', 'NumMosquitos', 'spray_dist', 'spray_status', 'Year',
       'nearest_station', 'date_lag', 'Sunset', 'DaylightHrs', 'Tmax', 'Tmin',
       'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Cool', 'PrecipTotal',
       'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed',
       'Station'],
      dtype='object')

In [54]:
full_condensed['Month'] = full_condensed.Date.map(lambda x : x.month)

In [61]:
cols = ['Month','Species','Street','NumMosquitos','spray_dist', 
        'spray_status', 'Year','Sunset', 'DaylightHrs',
        'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Cool',
        'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed',
        'ResultDir', 'AvgSpeed']
        
        # Note: in the regression problem I kept the "Year" feature - this is different from what we did
        # for the kaggle classification problem, since our train/test split are created by ourselves, meaning
        # they should have a similar mix of records from all years. Therefore it is possible that Year could  
        # be inferred for the impact on number of mosquitos.
        
        # Additional features added in the version2:
        
        #'WetWeather', 'RH', 'CoolYearlyCum', 'Sunset7dAvg', 'DaylightHrs7dAvg', 'Tmax7dAvg', 'Tmin7dAvg',
        #'Tavg7dAvg', 'Depart7dAvg', 'DewPoint7dAvg', 'WetBulb7dAvg',
        #'Cool7dAvg', 'PrecipTotal7dAvg', 'StnPressure7dAvg', 'SeaLevel7dAvg',
        #'ResultSpeed7dAvg', 'ResultDir7dAvg', 'AvgSpeed7dAvg'

In [62]:
full_condensed = full_condensed[cols]

In [63]:
full_condensed.dtypes

Month             int64
Species          object
Street           object
NumMosquitos      int64
spray_dist      float64
spray_status      int64
Year              int64
Sunset          float64
DaylightHrs     float64
Tmax            float64
Tmin            float64
Tavg            float64
Depart          float64
DewPoint        float64
WetBulb         float64
Cool            float64
PrecipTotal     float64
StnPressure     float64
SeaLevel        float64
ResultSpeed     float64
ResultDir       float64
AvgSpeed        float64
dtype: object

## Train/Test split

In [65]:
y = full_condensed['NumMosquitos']
X = full_condensed[[col for col in full_condensed.columns if col != 'NumMosquitos']]

## Export to CSV

In [66]:
X.to_csv('data/X_regression.csv',index=False)
y.to_csv('data/y_regression.csv',index=False, header='NumMosquitos')